### Clean column by column

In [78]:
import pandas as pd
from utils import save_csv

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

filename = "main_dataset_tcr_phla.csv"
main_df = pd.read_csv(datapath+filename, sep=',')
print(len(main_df))
print(main_df.columns)

193800
Index(['antigen.epitope', 'mhc.allele', 'mhc.class', 'alpha.v', 'alpha.j',
       'alpha.cdr1', 'alpha.cdr2', 'alpha.cdr3', 'alpha.vseq', 'beta.v',
       'beta.d', 'beta.j', 'beta.cdr1', 'beta.cdr2', 'beta.cdr3', 'beta.vseq',
       'dataset.tag'],
      dtype='object')


#### 1. mhc.class == I

In [79]:
main_df = main_df[main_df['mhc.class']=='MHCI']
print(len(main_df))

187795


#### 2. antigen.epitope

In [80]:
import re
def check_uppercase(text):
    pattern = "^[A-Z]+$"
    return bool(re.match(pattern, text))

main_df['check'] = main_df['antigen.epitope'].apply(check_uppercase)
print(main_df[main_df['check']==False]['antigen.epitope'])
main_df = main_df[main_df['check']==True].reset_index(drop=True)
print(f"number of checked epitope data: {len(main_df)}")

16         LLFGFPVYV + SCM(F5)
34         LLFGKPVYV + SCM(K5)
40         LLFGFPVFV + SCM(F5)
41         LLFGFAVYV + SCM(F5)
43         LLFGFPVYV + SCM(F5)
3424        MLYQHLLPL + OX(M1)
3425        MLYQHLLPL + OX(M1)
6009        MLYQHLLPL + OX(M1)
126622    ILDQVPFSV + NLeu(L2)
127086    LLSYFGTPT + METH(Y4)
127088     LLSYFGTPT + SCM(Y4)
128055    YMNGTMSQV + DEAM(N3)
Name: antigen.epitope, dtype: object
number of checked epitope data: 187783


#### 3. mhc.allele -> hla.allele

In [81]:
main_df['mhc.allele'].unique()

array(['HLA-E*01:01', 'HLA-E*01:03', 'HLA-B8', 'HLA-B*44:05',
       'HLA-B*08:01', 'HLA-B*44:02', 'HLA-A*02:01', 'HLA-A*02:01 K66A',
       'E63Q mutant', 'HLA-A2', 'HLA-B*27:05', 'HLA-B*35:01',
       'HLA-B*44:03', 'HLA-A*02:01 K66A mutant',
       'HLA-A*02:01 A150P mutant', 'HLA-B*35:08', 'HLA-B*35:08 Q65A',
       'T69A', 'Q155A mutant', 'HLA-B*57:01', 'HLA-B*57:03',
       'HLA-A*24:02', 'HLA-B*51:01', 'HLA-A*01:01', 'HLA-A11',
       'HLA-B*07:02', 'HLA-B7', 'HLA-A*11:01', 'HLA-A*02:01 A69G mutant',
       'HLA-A*02:01 E166A mutant', 'HLA-A*02:01 R65A mutant',
       'HLA-A*02:01 T163A mutant', 'HLA-A*02:01 W167A mutant',
       'HLA-B*35:08 Q155A mutant', '(UNK)', 'HLA-B*15:01', 'HLA-B27',
       'HLA-A1', 'HLA-B*38:01', 'HLA-C*14:02', 'HLA-B57', 'HLA-B18',
       'HLA-Cw3', 'HLA-B*37:01', 'HLA-C*07:02', 'HLA-B35', 'HLA-A*68:01',
       'HLA-A*30:01', 'HLA-A*03:01', 'HLA-B*40:01', 'HLA-B*18:01',
       'HLA-C*06:02', 'HLA-C*07:01', 'HLA-C*05:01', 'HLA-A*80:01',
       'HLA-A*3

In [82]:
for mhc in main_df['mhc.allele'].unique():
    if 'HLA' not in mhc:
        print(mhc)

E63Q mutant
T69A
Q155A mutant
(UNK)
H-2Kb


In [83]:
main_df = main_df[~main_df['mhc.allele'].isin(['H-2Kb'])]
print(f"number of correct mhc name data: {len(main_df)}")

main_df['mhc.allele'].replace({
    'E63Q mutant': 'HLA-A*02:01 E63Q mutant',
    'T69A': 'HLA-B*35:08 T69A',
    'Q155A mutant': 'HLA-B*35:08 Q155A mutant',
}, inplace=True)

main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of data: {len(main_df)}")

number of correct mhc name data: 187782
number of data: 187781


In [84]:
for mhc in main_df['mhc.allele'].unique():
    if '*' not in mhc:
        print(mhc)

HLA-B8
HLA-A2
HLA-A11
HLA-B7
(UNK)
HLA-B27
HLA-A1
HLA-B57
HLA-B18
HLA-Cw3
HLA-B35
HLA-A2:01


In [85]:
main_df = main_df[~main_df['mhc.allele'].isin(['HLA-Cw3'])]
print(f"number of data: {len(main_df)}")

main_df['mhc.allele'].replace({
    'HLA-B8': 'HLA-B*08',
    'HLA-A2': 'HLA-A*02',
    'HLA-A11': 'HLA-A*11',
    'HLA-B7': 'HLA-B*07',
    'HLA-B27': 'HLA-B*27',
    'HLA-A1': 'HLA-A*01',
    'HLA-B57': 'HLA-B*57',
    'HLA-B18': 'HLA-B*18',
    'HLA-B35': 'HLA-B*35',
    'HLA-A*2:01': 'HLA-A*02:01',
    'HLA-B*8': 'HLA-B*08',
    'HLA-A2:01': 'HLA-A*02:01',
    # 'HLA-Cw3': 'HLA-C*03',
    'HLA-Cw* 16:01': 'HLA-C*16:01',
    'HLA-A*011': 'HLA-A*11',
}, inplace=True)

main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of data: {len(main_df)}")

number of data: 187780
number of data: 187774


In [86]:
for mhc in main_df['mhc.allele'].unique():
    if len(mhc.split(' ')) > 1:
        print(mhc)

HLA-A*02:01 K66A
HLA-A*02:01 E63Q mutant
HLA-A*02:01 K66A mutant
HLA-A*02:01 A150P mutant
HLA-B*35:08 Q65A
HLA-B*35:08 T69A
HLA-B*35:08 Q155A mutant
HLA-A*02:01 A69G mutant
HLA-A*02:01 E166A mutant
HLA-A*02:01 R65A mutant
HLA-A*02:01 T163A mutant
HLA-A*02:01 W167A mutant


In [87]:
def clear_mutant(data):
    if len(data.split(' ')) > 1:
        return ' '.join(data.split(' ')[:2])
    else:
        return data

main_df['mhc.allele'] = main_df['mhc.allele'].apply(clear_mutant)
main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of data: {len(main_df)}")

number of data: 187773


In [88]:
main_df = main_df.rename(columns={'mhc.allele':'hla.allele'})

#### 4. cdr3

In [90]:
import re
def check_uppercase(text):
    if text == '(UNK)':
        return True
    else:
        pattern = "^[A-Z]+$"
        return bool(re.match(pattern, text))

main_df['check'] = main_df['beta.cdr3'].apply(check_uppercase)
print(main_df[main_df['check']==False]['beta.cdr3'])
main_df = main_df[main_df['check']==True].reset_index(drop=True)
print(f"number of data: {len(main_df)}")

177624    CAASTSAVLGKKGSQETQYf
177633            CASEDFKNIQYf
177646          CASSDGQGRLGYTf
177655         CASSGFRDRVNEQYf
177680          CASSPMDLLDEQYf
177694     CASSQGPGERAGFNYEQYf
177715           CASYNFGQYGYTf
177724        CSARGWVSNNQETQYf
178442       CAGSLALRLADQETQYf
178444           CAIRTSGDYEQYf
178454       CASGLALRGGDQETQYf
178471           CASRGSGVYEQYf
178486           CASRTSGDYEQYf
178490      CASSDNPLVGGFTDMQYf
178496           CASSFDKGyEQYF
178499       CASSFLTGARSKNIQYf
178543           CASSLGPRyEQYF
178611        CASSPRDFSPTDTQYf
178671         CASSVDKGGADTQYf
178694         CATRDRDRENQPQHf
178729         CSGGGYGSGTYEQYf
178733           CSVEGMRDYGYTf
178748        RASSSLRGTAADTQYf
187726                 Donor13
Name: beta.cdr3, dtype: object
number of data: 187749


- for alpha.cdr3
    - CAGLGGT **#** GGSNYKLTF, CAVN **?** PPFGNEKLTF -> (UNK) 
    - Because of the paired beta.cdr3 data

In [91]:
main_df['alpha.cdr3'].replace({
    'CAGLGGT#GGSNYKLTF': '(UNK)',
    'CAVN?PPFGNEKLTF': '(UNK)',
}, inplace=True)

main_df['check'] = main_df['alpha.cdr3'].apply(check_uppercase)
print(main_df[main_df['check']==False]['alpha.cdr3'])
main_df = main_df[main_df['check']==True].reset_index(drop=True)
print(f"number of data: {len(main_df)}")

187200    CAGTHGGSQGSLIf
Name: alpha.cdr3, dtype: object
number of data: 187748


- peptide-cdr3b pair: fundemental

In [92]:
main_df = main_df[main_df['beta.cdr3']!='(UNK)'].reset_index(drop=True)
print(f"number of data: {len(main_df)}")

number of data: 174259


- other cdrs

In [93]:
cdrs = ['alpha.cdr1', 'alpha.cdr2', 'beta.cdr1', 'beta.cdr2']

for cdr in cdrs:
    main_df['check'] = main_df[cdr].apply(check_uppercase)
    print(main_df[main_df['check']==False][cdr])
    main_df = main_df[main_df['check']==True].reset_index(drop=True)
    print(f"check {cdr}, number of remaining data: {len(main_df)}")

Series([], Name: alpha.cdr1, dtype: object)
check alpha.cdr1, number of remaining data: 174259
Series([], Name: alpha.cdr2, dtype: object)
check alpha.cdr2, number of remaining data: 174259
Series([], Name: beta.cdr1, dtype: object)
check beta.cdr1, number of remaining data: 174259
Series([], Name: beta.cdr2, dtype: object)
check beta.cdr2, number of remaining data: 174259


- save for preparing level II/III

In [94]:
main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of duplicated data: {len(main_df)}")

savepath = to_your_path + "TCRdata/collect/"
savename = "main_dataset_tcr_phla_cleaned_1.csv"
save_csv(main_df, savepath, savename)

number of duplicated data: 174259
File /data/lujd/TCRdata/collect/main_dataset_tcr_phla_cleaned_1.csv has been successfully saved.


---

#### 5. gene

- alpha.v

In [55]:
import pandas as pd
from utils import save_csv

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

filename = "main_dataset_tcr_phla_cleaned_1.csv"
main_df = pd.read_csv(datapath+filename, sep=',')
print(len(main_df))
print(main_df.columns)

174259
Index(['antigen.epitope', 'hla.allele', 'mhc.class', 'alpha.v', 'alpha.j',
       'alpha.cdr1', 'alpha.cdr2', 'alpha.cdr3', 'alpha.vseq', 'beta.v',
       'beta.d', 'beta.j', 'beta.cdr1', 'beta.cdr2', 'beta.cdr3', 'beta.vseq',
       'dataset.tag', 'check'],
      dtype='object')


In [56]:
test_df = main_df
main_df['alpha.v'].unique()

array(['TRAV26-1*01', '(UNK)', 'TRAV21*01', 'TRAV12-2*01', 'TRAV19*01',
       'TRAV20*01', 'TRAV5*01', 'TRAV12-1*01', 'TRAV3*01', 'TRAV12-3*01',
       'TRAV39*01', 'TRAV8-3*01', 'TRAV22*01', 'TRAV17*01', 'TRAV35*01',
       'TRAV9-2*01', 'TRAV27*01', 'TRAV24*01', 'TRAV26-2*01',
       'TRAV29/DV5*01', 'TRAV30*01', 'TRAV14/DV4*01', 'TRAV14',
       'TRAV1-2*01', 'TRAV4*01', 'TRAV13-1*01', 'TRAV38-2/DV8*01',
       'TRAV1-1*01', 'TRAV1-2*03', 'TRAV14/DV4*02', 'TRAV8-1*01',
       'TRAV16*01', 'TRAV6*01', 'TRAV10*02', 'TRBV10-2*02', 'TRAV8-2*01',
       'TRAV8-6*02', 'TRAV25*01', 'TRAV8-4*01', 'TRAV2*01', 'TRAV26-1*02',
       'TRAV38-1*01', 'TRAV13-2*01', 'TRAV34*01', 'TRAV10*01',
       'TRAV23/DV6*01', 'TRAV36/DV7*01', 'TRAV41*01', 'TRAV8-6*01',
       'TRAV18*01', 'TRAV40*01', 'TRAV7*01', 'TRAV9-1*01', 'TRAV3',
       'TRAV23/DV6', 'TRAV21', 'TRAV5', 'TRAV14/DV4', 'TRAV13-1',
       'TRAV12-1', 'TRAV4', 'TRAV30', 'TRAV8-4', 'TRAV1-1', 'TRAV10',
       'TRAV8-6', 'TRAV2', 'TRAV8-2', 

In [57]:
clean_trav_dict = {}
for origin_trav in main_df['alpha.v'].unique():
    print_tag = 1
    if ("TRAV" not in origin_trav) and ("TRA" in origin_trav):
        print(origin_trav)
        trav = origin_trav.replace("TRA", "TRAV").strip()
    else:
        trav = origin_trav.strip()

    if trav == '(UNK)':
        clean_trav_dict[origin_trav] = trav
    elif ("," in trav) or ("?" in trav) or ("TRAV-" in trav) or ("TRB" in trav):
        clean_trav_dict[origin_trav] = '(UNK)'
    elif ":0" in trav:
        clean_trav_dict[origin_trav] = trav.replace(":0", "*0")
    elif "-0" in trav:
        clean_trav_dict[origin_trav] = trav.replace("-0", "*0")
    elif "-*0" in trav:
        clean_trav_dict[origin_trav] = trav.replace("-*0", "*0")
    elif "V0" in trav:
        clean_trav_dict[origin_trav] = trav.replace("V0", "V")
    elif " F" in trav:
        clean_trav_dict[origin_trav] = trav.replace(" F", "")
    else:
        clean_trav_dict[origin_trav] = trav
        print_tag = 0
    if print_tag:
        print(origin_trav, trav, clean_trav_dict[origin_trav])

(UNK) (UNK) (UNK)
TRBV10-2*02 TRBV10-2*02 (UNK)
TRAV1-01 TRAV1-01 TRAV1*01
TRAV12-2, TRAV21 TRAV12-2, TRAV21 (UNK)
TRAV2-01 TRAV2-01 TRAV2*01
TRAV35:01 TRAV35:01 TRAV35*01
TRAV12-3:01 TRAV12-3:01 TRAV12-3*01
TRAV25:01 TRAV25:01 TRAV25*01
TRAV3-01 TRAV3-01 TRAV3*01
TRAV10-01 TRAV10-01 TRAV10*01
TRAV1-2:01 TRAV1-2:01 TRAV1-2*01
TRAV14/DV4:01 TRAV14/DV4:01 TRAV14/DV4*01
TRAV29/DV5:01 TRAV29/DV5:01 TRAV29/DV5*01
TRAV1-1:01 TRAV1-1:01 TRAV1-1*01
TRAV6-01 TRAV6-01 TRAV6*01
TRAV38-2/DV8:01 TRAV38-2/DV8:01 TRAV38-2/DV8*01
TRAV13-2:01 TRAV13-2:01 TRAV13-2*01
TRAV5-01 TRAV5-01 TRAV5*01
TRAV23/DV6:01 TRAV23/DV6:01 TRAV23/DV6*01
TRAV9-2:01 TRAV9-2:01 TRAV9-2*01
TRAV41:01 TRAV41:01 TRAV41*01
TRAV38-1:01 TRAV38-1:01 TRAV38-1*01
TRAV14/DV4:02 TRAV14/DV4:02 TRAV14/DV4*02
TRAV8-1:01 TRAV8-1:01 TRAV8-1*01
TRAV12-2:01 TRAV12-2:01 TRAV12-2*01
TRAV22:01 TRAV22:01 TRAV22*01
TRAV8-6:02 TRAV8-6:02 TRAV8-6*02
TRAV8-6:01 TRAV8-6:01 TRAV8-6*01
TRAV8-3:01 TRAV8-3:01 TRAV8-3*01
TRAV13-1:01 TRAV13-1:01 TRAV13-1*01


In [58]:
set(clean_trav_dict.values())

{'(UNK)',
 'TRAV1',
 'TRAV1*01',
 'TRAV1-1',
 'TRAV1-1*01',
 'TRAV1-2',
 'TRAV1-2*01',
 'TRAV1-2*03',
 'TRAV1-4',
 'TRAV10',
 'TRAV10*01',
 'TRAV10*02',
 'TRAV11',
 'TRAV12*02',
 'TRAV12-1',
 'TRAV12-1*01',
 'TRAV12-2',
 'TRAV12-2*01',
 'TRAV12-3',
 'TRAV12-3*01',
 'TRAV12-4',
 'TRAV121',
 'TRAV13-1',
 'TRAV13-1*01',
 'TRAV13-2',
 'TRAV13-2*01',
 'TRAV14',
 'TRAV14/DV4',
 'TRAV14/DV4*01',
 'TRAV14/DV4*02',
 'TRAV15',
 'TRAV16',
 'TRAV16*01',
 'TRAV17',
 'TRAV17*01',
 'TRAV18',
 'TRAV18*01',
 'TRAV19',
 'TRAV19*01',
 'TRAV2',
 'TRAV2*01',
 'TRAV2*02',
 'TRAV2-1',
 'TRAV2-2',
 'TRAV20',
 'TRAV20*01',
 'TRAV21',
 'TRAV21*01',
 'TRAV21-1',
 'TRAV21-2',
 'TRAV22',
 'TRAV22*01',
 'TRAV23',
 'TRAV23/DV6',
 'TRAV23/DV6*01',
 'TRAV24',
 'TRAV24*01',
 'TRAV25',
 'TRAV25*01',
 'TRAV251',
 'TRAV26',
 'TRAV26-1',
 'TRAV26-1*01',
 'TRAV26-1*02',
 'TRAV26-1*03',
 'TRAV26-2',
 'TRAV26-2*01',
 'TRAV27',
 'TRAV27*01',
 'TRAV28',
 'TRAV29',
 'TRAV29-1',
 'TRAV29/DV5',
 'TRAV29/DV5*01',
 'TRAV29/DV5*02',


In [60]:
main_df['alpha.v'] = main_df['alpha.v'].apply(lambda x: clean_trav_dict[x])
main_df['alpha.v'].nunique()

127

- alpha.j

In [61]:
test_df = main_df
main_df['alpha.j'].unique()

array(['TRAJ37*01', '(UNK)', 'TRAJ6*01', 'TRAJ13*01', 'TRAJ24*02',
       'TRAJ34*01', 'TRAJ27*01', 'TRAJ53*01', 'TRAJ31*01', 'TRAJ23*01',
       'TRAJ29*01', 'TRAJ12*01', 'TRAJ33*01', 'TRAJ28*01', 'TRAJ40*01',
       'TRAJ22*01', 'TRAJ49*01', 'TRAJ43*01', 'TRAJ42*01', 'TRAJ52*01',
       'TRAJ54*01', 'TRAJ39*01', 'TRAJ21*01', 'TRAJ41*01', 'TRAJ8*01',
       'TRDJ1*01', 'TRAJ23*02', 'TRAJ11*01', 'TRDJ2*01', 'TRDJ4*01',
       'TRAJ26*01', 'TRAJ48*01', 'TRAJ9*01', 'TRAJ15*01', 'TRAJ3*01',
       'TRAJ30*01', 'TRAJ44*01', 'TRAJ5*01', 'TRAJ13*02', 'TRBJ1-1*01',
       'TRAJ20*01', 'TRAJ32*02', 'TRAJ24*03', 'TRAJ18*01', 'TRAJ38*01',
       'TRAJ45*01', 'TRAJ10*01', 'TRDJ3*01', 'TRAJ7*01', 'TRAJ57*01',
       'TRAJ17*01', 'TRAJ24*01', 'TRAJ50*01', 'TRAJ47*01', 'TRAJ4*01',
       'TRAJ56*01', 'TRAJ36*01', 'TRAJ32*01', 'TRAJ46*01', 'TRAJ14*01',
       'TRAJ34', 'TRAJ45', 'TRAJ28', 'TRAJ14', 'TRAJ27', 'TRAJ9', 'TRAJ8',
       'TRAJ40', 'TRAJ13', 'TRAJ60', 'TRAJ35', 'TRAJ4', 'TRAJ42',
       'T

In [73]:
import re
clean_traj_dict = {}
for origin_traj in main_df['alpha.j'].unique():
    print_tag = 1
    if ("TRAJ" not in origin_traj) and ("TRA" in origin_traj):
        print(origin_traj)
        traj = origin_traj.replace("TRA", "TRAJ").strip()
    else:
        traj = origin_traj.strip()

    if traj == '(UNK)':
        clean_traj_dict[origin_traj] = traj
    elif ("," in traj) or ("?" in traj) or ("TRAJ-" in traj) or ("TRA" not in traj) or bool(re.match(r"^[A-Z]*$", traj)):
        clean_traj_dict[origin_traj] = '(UNK)'
    elif ":0" in traj:
        clean_traj_dict[origin_traj] = traj.replace(":0", "*0")
    elif "-0" in traj:
        clean_traj_dict[origin_traj] = traj.replace("-0", "*0")
    elif "-*0" in traj:
        clean_traj_dict[origin_traj] = traj.replace("-*0", "*0")
    elif "J0" in traj:
        clean_traj_dict[origin_traj] = traj.replace("J0", "J")
    elif "F" in traj:
        clean_traj_dict[origin_traj] = traj.replace("F", "")
    else:
        clean_traj_dict[origin_traj] = traj
        print_tag = 0
    if print_tag:
        print(origin_traj, traj, clean_traj_dict[origin_traj])

(UNK) (UNK) (UNK)
TRDJ1*01 TRDJ1*01 (UNK)
TRDJ2*01 TRDJ2*01 (UNK)
TRDJ4*01 TRDJ4*01 (UNK)
TRBJ1-1*01 TRBJ1-1*01 (UNK)
TRDJ3*01 TRDJ3*01 (UNK)
TRAJ4-01 TRAJ4-01 TRAJ4*01
TRAJ5-01 TRAJ5-01 TRAJ5*01
TRAJ3-01 TRAJ3-01 TRAJ3*01
TRAJ1-01 TRAJ1-01 TRAJ1*01
TRAJ24:02 TRAJ24:02 TRAJ24*02
TRAJ2-01 TRAJ2-01 TRAJ2*01
TRAJ9-01 TRAJ9-01 TRAJ9*01
TRAJ6-01 TRAJ6-01 TRAJ6*01
TRAJ8-01 TRAJ8-01 TRAJ8*01
TRAJ13:02 TRAJ13:02 TRAJ13*02
TRAJ49:01 TRAJ49:01 TRAJ49*01
TRAJ20:01 TRAJ20:01 TRAJ20*01
TRAJ42:01 TRAJ42:01 TRAJ42*01
TRAJ24:01 TRAJ24:01 TRAJ24*01
TRAJ21:01 TRAJ21:01 TRAJ21*01
TRAJ53:01 TRAJ53:01 TRAJ53*01
TRAJ17:01 TRAJ17:01 TRAJ17*01
TRAJ54:01 TRAJ54:01 TRAJ54*01
TRAJ39:01 TRAJ39:01 TRAJ39*01
TRAJ TRAJ (UNK)
TRAJ53:02 TRAJ53:02 TRAJ53*02
TRAJ36:01 TRAJ36:01 TRAJ36*01
TRAJ41:01 TRAJ41:01 TRAJ41*01
TRAJ16:01 TRAJ16:01 TRAJ16*01
TRAJ31:01 TRAJ31:01 TRAJ31*01
TRAJ38:01 TRAJ38:01 TRAJ38*01
TRAJ37:01 TRAJ37:01 TRAJ37*01
TRAJ48:01 TRAJ48:01 TRAJ48*01
TRAJ9:01 TRAJ9:01 TRAJ9*01
TRAJ26:01 TRAJ26:01 TRAJ26*01

In [74]:
set(clean_traj_dict.values())

{'(UNK)',
 'TRAJ1*01',
 'TRAJ10',
 'TRAJ10*01',
 'TRAJ11',
 'TRAJ11*01',
 'TRAJ12',
 'TRAJ12*01',
 'TRAJ13',
 'TRAJ13*01',
 'TRAJ13*02',
 'TRAJ14',
 'TRAJ14*01',
 'TRAJ15',
 'TRAJ15*01',
 'TRAJ16',
 'TRAJ16*01',
 'TRAJ17',
 'TRAJ17*01',
 'TRAJ18',
 'TRAJ18*01',
 'TRAJ2',
 'TRAJ2*01',
 'TRAJ2-1',
 'TRAJ20',
 'TRAJ20*01',
 'TRAJ20-1',
 'TRAJ21',
 'TRAJ21*01',
 'TRAJ21-1',
 'TRAJ22',
 'TRAJ22*01',
 'TRAJ23',
 'TRAJ23*01',
 'TRAJ23*02',
 'TRAJ24',
 'TRAJ24*01',
 'TRAJ24*02',
 'TRAJ24*03',
 'TRAJ26',
 'TRAJ26*01',
 'TRAJ27',
 'TRAJ27*01',
 'TRAJ28',
 'TRAJ28*01',
 'TRAJ29',
 'TRAJ29*01',
 'TRAJ3',
 'TRAJ3*01',
 'TRAJ3-1',
 'TRAJ30',
 'TRAJ30*01',
 'TRAJ31',
 'TRAJ31*01',
 'TRAJ31-1',
 'TRAJ32',
 'TRAJ32*01',
 'TRAJ32*02',
 'TRAJ32-1',
 'TRAJ33',
 'TRAJ33*01',
 'TRAJ33-1',
 'TRAJ34',
 'TRAJ34*01',
 'TRAJ34-1',
 'TRAJ35',
 'TRAJ36',
 'TRAJ36*01',
 'TRAJ36/DV7*02',
 'TRAJ37',
 'TRAJ37*01',
 'TRAJ37*02',
 'TRAJ38',
 'TRAJ38*01',
 'TRAJ39',
 'TRAJ39*01',
 'TRAJ4',
 'TRAJ4*01',
 'TRAJ40',
 'TRAJ4

In [75]:
main_df['alpha.j'] = main_df['alpha.j'].apply(lambda x: clean_traj_dict[x])
main_df['alpha.j'].nunique()

122

In [76]:
main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of duplicated data: {len(main_df)}")

savepath = to_your_path + "TCRdata/collect/"
savename = "main_dataset_tcr_phla_cleaned_2.csv"
save_csv(main_df, savepath, savename)

number of duplicated data: 174254
File /data/lujd/TCRdata/collect/main_dataset_tcr_phla_cleaned_2.csv has been successfully saved.


- beta.v

In [15]:
import pandas as pd
from utils import save_csv

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

filename = "main_dataset_tcr_phla_cleaned_2.csv"
main_df = pd.read_csv(datapath+filename, sep=',')
print(len(main_df))
print(main_df.columns)

174254
Index(['antigen.epitope', 'hla.allele', 'mhc.class', 'alpha.v', 'alpha.j',
       'alpha.cdr1', 'alpha.cdr2', 'alpha.cdr3', 'alpha.vseq', 'beta.v',
       'beta.d', 'beta.j', 'beta.cdr1', 'beta.cdr2', 'beta.cdr3', 'beta.vseq',
       'dataset.tag', 'check'],
      dtype='object')


In [16]:
main_df['beta.v'].unique()

array(['TRBV14*01', 'TRBV7-8*01', 'TRBV6-5*01', 'TRBV5-1*01',
       'TRBV7-9*01', 'TRBV6-1*01', 'TRBV30*02', 'TRBV9*01', 'TRBV9*02',
       'TRBV20-1*01', 'TRBV6-3*01', 'TRBV12-4*01', 'TRBV19*01',
       'TRBV5-6*01', 'TRBV4-1*01', 'TRBV27*01', 'TRBV7-3*01',
       'TRBV6-4*01', 'TRBV10-3*01', 'TRBV11-2*01', 'TRBV7-2*02',
       'TRBV7-6*01', '(UNK)', 'TRBV25-1*01', 'TRBV28*01', 'TRBV10-2*02',
       'TRBV4-3*01', 'TRBV18*01', 'TRBV5-4*01', 'TRBV12-3*01',
       'TRBV7-9*03', 'TRBV6-6*02', 'TRBV2*01', 'TRBV7-2*01', 'TRBV3-1*01',
       'TRBV15*02', 'TRBV24-1*01', 'TRBV6-6*01', 'TRBV5-5*01',
       'TRBV13*01', 'TRBV4-2*01', 'TRBV16*01', 'TRBV10-1*01',
       'TRBV7-7*01', 'TRBV11-3*01', 'TRBV6-2*01', 'TRBV15*01',
       'TRBV11-1*01', 'TRBV30*01', 'TRBV29-1*01', 'TRBV10-2*01',
       'TRBV12-5*01', 'TRBV7-2*03', 'TRBV5-8*01', 'TRBV7-4*01',
       'TRBV6-9*01', 'TRBV6-8*01', 'TRBV10-3*02', 'TRBV20-1*02',
       'TRBV7-8*02', 'TRBV19*02', 'TRBV6-4*02', 'TRBV27', 'TRBV4',
       'TRBV12'

In [17]:
import re

clean_trbv_dict = {}
for origin_trbv in main_df['beta.v'].unique():
    print_tag = 1
    if ("TRBV" not in origin_trbv) and ("TRB" in origin_trbv):
        print(origin_trbv)
        trbv = origin_trbv.replace("TRB", "TRBV").strip()
    else:
        trbv = origin_trbv.strip()

    if trbv == '(UNK)':
        clean_trbv_dict[origin_trbv] = trbv
    elif ("?" in trbv) or ("TRBV-" in trbv) or ("TRB" not in trbv) or bool(re.match(r"^[A-Z]*$", trbv)):
        clean_trbv_dict[origin_trbv] = '(UNK)'
    elif ":0" in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace(":0", "*0")
    elif "-0" in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace("-0", "-")
    elif "-*0" in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace("-*0", "*0")
    elif "V0" in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace("V0", "V")
    elif " F" in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace(" F", "")
    else:
        clean_trbv_dict[origin_trbv] = trbv
        print_tag = 0
    if print_tag:
        print(origin_trbv, trbv, clean_trbv_dict[origin_trbv])

(UNK) (UNK) (UNK)
TRBV11-01 TRBV11-01 TRBV11-1
TRBV7-08 TRBV7-08 TRBV7-8
TRBV25-01 TRBV25-01 TRBV25-1
TRBV20-01 TRBV20-01 TRBV20-1
TRBV29-01 TRBV29-01 TRBV29-1
TRBV6-01 TRBV6-01 TRBV6-1
TRBV12-03 TRBV12-03 TRBV12-3
TRBV2-01 TRBV2-01 TRBV2-1
TRBV21-03 TRBV21-03 TRBV21-3
TRBV6-04 TRBV6-04 TRBV6-4
TRBV6-05 TRBV6-05 TRBV6-5
TRBV3-01 TRBV3-01 TRBV3-1
TRBV4-01 TRBV4-01 TRBV4-1
TRBV18-01 TRBV18-01 TRBV18-1
TRBV7-03 TRBV7-03 TRBV7-3
TRBV16-01 TRBV16-01 TRBV16-1
TRBV1-01 TRBV1-01 TRBV1-1
TRBV6-03 TRBV6-03 TRBV6-3
TRBV6-02 TRBV6-02 TRBV6-2
TRBV10-02 TRBV10-02 TRBV10-2
TRBV12-04 TRBV12-04 TRBV12-4
TRBV7-06 TRBV7-06 TRBV7-6
TRBV7-02 TRBV7-02 TRBV7-2
TRBV6-02/3 TRBV6-02/3 TRBV6-2/3
TRBV7-09 TRBV7-09 TRBV7-9
TRBV5-04 TRBV5-04 TRBV5-4
TRBV5-01 TRBV5-01 TRBV5-1
TRBV5-06 TRBV5-06 TRBV5-6
TRBV5-08 TRBV5-08 TRBV5-8
TRBV6-06 TRBV6-06 TRBV6-6
TRBV24-01 TRBV24-01 TRBV24-1
TRBV10-03 TRBV10-03 TRBV10-3
TRBV11-02 TRBV11-02 TRBV11-2
TRBV28-01 TRBV28-01 TRBV28-1
TRBV4-02 TRBV4-02 TRBV4-2
TRBV5-05 TRBV5-05 TRBV5-

In [18]:
set(clean_trbv_dict.values())

{'(UNK)',
 'TRBV1',
 'TRBV1-1',
 'TRBV1-1*01',
 'TRBV1-4',
 'TRBV1-4*01',
 'TRBV1-5',
 'TRBV10',
 'TRBV10-1',
 'TRBV10-1*01',
 'TRBV10-2',
 'TRBV10-2*01',
 'TRBV10-2*02',
 'TRBV10-3',
 'TRBV10-3*01',
 'TRBV10-3*02',
 'TRBV11-1',
 'TRBV11-1*01',
 'TRBV11-2',
 'TRBV11-2*01',
 'TRBV11-3',
 'TRBV11-3*01',
 'TRBV12',
 'TRBV12-2',
 'TRBV12-3',
 'TRBV12-3*01',
 'TRBV12-3,12-4',
 'TRBV12-3/12-4',
 'TRBV12-3/4',
 'TRBV12-4',
 'TRBV12-4*01',
 'TRBV12-5',
 'TRBV12-5*01',
 'TRBV13',
 'TRBV13*01',
 'TRBV13-2',
 'TRBV13-6',
 'TRBV14',
 'TRBV14*01',
 'TRBV14-1',
 'TRBV15',
 'TRBV15*01',
 'TRBV15*02',
 'TRBV16',
 'TRBV16*01',
 'TRBV16-1',
 'TRBV17',
 'TRBV17*01',
 'TRBV18',
 'TRBV18*01',
 'TRBV18-1',
 'TRBV19',
 'TRBV19*01',
 'TRBV19*02',
 'TRBV19*03',
 'TRBV19*04',
 'TRBV19*05',
 'TRBV19-1',
 'TRBV2',
 'TRBV2*01',
 'TRBV2-1',
 'TRBV2-1*01',
 'TRBV2-2*01',
 'TRBV2-3',
 'TRBV2-5',
 'TRBV2-6*01',
 'TRBV2-7*01',
 'TRBV20',
 'TRBV20-1',
 'TRBV20-1*01',
 'TRBV20-1*02',
 'TRBV201-1',
 'TRBV21',
 'TRBV21-1',

In [19]:
main_df['beta.v'] = main_df['beta.v'].apply(lambda x: clean_trbv_dict[x])
main_df['beta.v'].nunique()

183

- beta.j

In [22]:
main_df['beta.j'].unique()

array(['TRBJ2-3*01', 'TRBJ2-7*01', 'TRBJ1-1*01', 'TRBJ2-1*01',
       'TRBJ2-2*01', 'TRBJ1-2*01', 'TRBJ2-4*01', 'TRBJ1-5*01',
       'TRBJ1-4*01', 'TRBJ2-5*01', '(UNK)', 'TRBJ1-3*01', 'TRBJ2-6*01',
       'TRBJ1-6*02', 'TRBJ1-6*01', 'TRBJ2-1', 'TRBJ2-2', 'TRBJ1-3',
       'TRBJ2-4', 'TRBJ1-5', 'TRBJ2-3', 'TRBJ2-7', 'TRBJ1-1', 'TRBJ1-2',
       'TRBJ1-4', 'TRBJ2-5', 'TRBJ2-6', 'TRBJ1-6', 'TRBJ2', 'TRBJ1-7',
       'TRBJ5-6', 'TRBJ2-7:01', 'TRBJ1-2:01', 'TRBJ2-1:01', 'TRBJ2-2:01',
       'TRBJ2-5:01', 'TRBJ2-3:01', 'TRBJ1-5:01', 'TRBJ1-1:01',
       'TRBJ2-4:01', 'TRBJ1-3:01', 'TRBJ1-4:01', 'TRBJ2-6:01', 'TRAJ30',
       'TRAJ26', 'TRBJ53:01', 'TRBJ54:01', 'TRBJ3-1:01', 'TRBJ42:01',
       'TRBJ39:01', 'TRBJ45:01', 'TRBJ1:01', 'TRBJ2:01', 'TRBJ31:01',
       'TRBJ1-6:02', 'TRBJ43:01', 'TRBJ33:01', 'TRBJ27:01', 'TRBJ37:02',
       'TRBJ8:01', 'TRBJ52:01', 'TRBJ9:01', 'TRBJ36:01', 'TRBJ2-7:02',
       'Donor 13', 'TRBJ5-1', 'TRBJ10-1', 'TRBJ20-1', 'TRBJ19-1',
       'TRBJ24-1', 'TRBJ21-1',

In [23]:
import re

clean_trbj_dict = {}
for origin_trbj in main_df['beta.j'].unique():
    print_tag = 1
    if ("TRBJ" not in origin_trbj) and ("TRB" in origin_trbj):
        print(origin_trbj)
        trbj = origin_trbj.replace("TRB", "TRBJ").strip()
    else:
        trbj = origin_trbj.strip()

    if trbj == '(UNK)':
        clean_trbj_dict[origin_trbj] = trbj
    elif ("?" in trbj) or ("TRBJ-" in trbj) or ("TRB" not in trbj) or bool(re.match(r"^[A-Z]*$", trbj)):
        clean_trbj_dict[origin_trbj] = '(UNK)'
    elif ":0" in trbj:
        clean_trbj_dict[origin_trbj] = trbj.replace(":0", "*0")
    elif "-0" in trbj:
        clean_trbj_dict[origin_trbj] = trbj.replace("-0", "-")
    elif "-*0" in trbj:
        clean_trbj_dict[origin_trbj] = trbj.replace("-*0", "*0")
    elif "J0" in trbj:
        clean_trbj_dict[origin_trbj] = trbj.replace("J0", "J")
    elif " F" in trbj:
        clean_trbj_dict[origin_trbj] = trbj.replace(" F", "")
    else:
        clean_trbj_dict[origin_trbj] = trbj
        print_tag = 0
    if print_tag:
        print(origin_trbj, trbj, clean_trbj_dict[origin_trbj])

(UNK) (UNK) (UNK)
TRBJ2-7:01 TRBJ2-7:01 TRBJ2-7*01
TRBJ1-2:01 TRBJ1-2:01 TRBJ1-2*01
TRBJ2-1:01 TRBJ2-1:01 TRBJ2-1*01
TRBJ2-2:01 TRBJ2-2:01 TRBJ2-2*01
TRBJ2-5:01 TRBJ2-5:01 TRBJ2-5*01
TRBJ2-3:01 TRBJ2-3:01 TRBJ2-3*01
TRBJ1-5:01 TRBJ1-5:01 TRBJ1-5*01
TRBJ1-1:01 TRBJ1-1:01 TRBJ1-1*01
TRBJ2-4:01 TRBJ2-4:01 TRBJ2-4*01
TRBJ1-3:01 TRBJ1-3:01 TRBJ1-3*01
TRBJ1-4:01 TRBJ1-4:01 TRBJ1-4*01
TRBJ2-6:01 TRBJ2-6:01 TRBJ2-6*01
TRAJ30 TRAJ30 (UNK)
TRAJ26 TRAJ26 (UNK)
TRBJ53:01 TRBJ53:01 TRBJ53*01
TRBJ54:01 TRBJ54:01 TRBJ54*01
TRBJ3-1:01 TRBJ3-1:01 TRBJ3-1*01
TRBJ42:01 TRBJ42:01 TRBJ42*01
TRBJ39:01 TRBJ39:01 TRBJ39*01
TRBJ45:01 TRBJ45:01 TRBJ45*01
TRBJ1:01 TRBJ1:01 TRBJ1*01
TRBJ2:01 TRBJ2:01 TRBJ2*01
TRBJ31:01 TRBJ31:01 TRBJ31*01
TRBJ1-6:02 TRBJ1-6:02 TRBJ1-6*02
TRBJ43:01 TRBJ43:01 TRBJ43*01
TRBJ33:01 TRBJ33:01 TRBJ33*01
TRBJ27:01 TRBJ27:01 TRBJ27*01
TRBJ37:02 TRBJ37:02 TRBJ37*02
TRBJ8:01 TRBJ8:01 TRBJ8*01
TRBJ52:01 TRBJ52:01 TRBJ52*01
TRBJ9:01 TRBJ9:01 TRBJ9*01
TRBJ36:01 TRBJ36:01 TRBJ36*01
TRBJ2-7:02 T

In [24]:
set(clean_trbj_dict.values())

{'(UNK)',
 'TRBJ1*01',
 'TRBJ1-1',
 'TRBJ1-1*01',
 'TRBJ1-2',
 'TRBJ1-2*01',
 'TRBJ1-3',
 'TRBJ1-3*01',
 'TRBJ1-4',
 'TRBJ1-4*01',
 'TRBJ1-5',
 'TRBJ1-5*01',
 'TRBJ1-6',
 'TRBJ1-6*01',
 'TRBJ1-6*02',
 'TRBJ1-7',
 'TRBJ10-1',
 'TRBJ17-1',
 'TRBJ19-1',
 'TRBJ2',
 'TRBJ2*01',
 'TRBJ2-1',
 'TRBJ2-1*01',
 'TRBJ2-2',
 'TRBJ2-2*01',
 'TRBJ2-3',
 'TRBJ2-3*01',
 'TRBJ2-4',
 'TRBJ2-4*01',
 'TRBJ2-5',
 'TRBJ2-5*01',
 'TRBJ2-6',
 'TRBJ2-6*01',
 'TRBJ2-7',
 'TRBJ2-7*01',
 'TRBJ2-7*02',
 'TRBJ20-1',
 'TRBJ21-1',
 'TRBJ24-1',
 'TRBJ27*01',
 'TRBJ3-1*01',
 'TRBJ31*01',
 'TRBJ33*01',
 'TRBJ36*01',
 'TRBJ37*02',
 'TRBJ38-2/DV8*01',
 'TRBJ39*01',
 'TRBJ42*01',
 'TRBJ43*01',
 'TRBJ45*01',
 'TRBJ5-1',
 'TRBJ5-6',
 'TRBJ52*01',
 'TRBJ53*01',
 'TRBJ54*01',
 'TRBJ8*01',
 'TRBJ9*01'}

In [25]:
main_df['beta.j'] = main_df['beta.j'].apply(lambda x: clean_trbj_dict[x])
main_df['beta.j'].nunique()

57

In [26]:
main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of duplicated data: {len(main_df)}")

savepath = to_your_path + "TCRdata/collect/"
savename = "main_dataset_tcr_phla_cleaned_2.csv"
save_csv(main_df, savepath, savename)

number of duplicated data: 174249
File /data/lujd/TCRdata/collect/main_dataset_tcr_phla_cleaned_2.csv has been successfully saved.


---

In [83]:
import pandas as pd
from utils import save_csv

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

filename = "main_dataset_tcr_phla_cleaned_2.csv"
main_df = pd.read_csv(datapath+filename, sep=',')
print(len(main_df))
print(main_df.columns)

174249
Index(['antigen.epitope', 'hla.allele', 'mhc.class', 'alpha.v', 'alpha.j',
       'alpha.cdr1', 'alpha.cdr2', 'alpha.cdr3', 'alpha.vseq', 'beta.v',
       'beta.d', 'beta.j', 'beta.cdr1', 'beta.cdr2', 'beta.cdr3', 'beta.vseq',
       'dataset.tag', 'check'],
      dtype='object')


In [84]:
clean_trbv_dict = {}
for origin_trbv in main_df['beta.v'].unique():
    print_tag = 1
    trbv = origin_trbv.strip()
    if "," in trbv:
        clean_trbv_dict[origin_trbv] = trbv.replace(",", "/")
    else:
        clean_trbv_dict[origin_trbv] = trbv
        print_tag = 0
    if print_tag:
        print(origin_trbv, trbv, clean_trbv_dict[origin_trbv])

TRBV6-2,6-3 TRBV6-2,6-3 TRBV6-2/6-3
TRBV12-3,12-4 TRBV12-3,12-4 TRBV12-3/12-4


In [85]:
main_df['beta.v'] = main_df['beta.v'].apply(lambda x: clean_trbv_dict[x])
main_df['beta.v'].nunique()

181

In [86]:
dup_df = main_df[main_df['beta.v']!='(UNK)']
dup_df = dup_df.drop_duplicates(ignore_index=True, keep='first')
print(len(dup_df))

dup_df = dup_df[dup_df['beta.j']!='(UNK)']
dup_df = dup_df.drop_duplicates(ignore_index=True, keep='first')
print(len(dup_df))

dup_df = dup_df[dup_df['alpha.v']!='(UNK)']
dup_df = dup_df.drop_duplicates(ignore_index=True, keep='first')
print(len(dup_df))

dup_df = dup_df[dup_df['alpha.j']!='(UNK)']
dup_df = dup_df.drop_duplicates(ignore_index=True, keep='first')
print(len(dup_df))

54193
52676
28579
27746


In [87]:
dup_df_mc = dup_df[dup_df['dataset.tag']=='mcpas']
dup_df_mc['beta.v'].unique()

array(['TRBV4', 'TRBV12', 'TRBV6', 'TRBV7', 'TRBV2', 'TRBV11-1',
       'TRBV7-8', 'TRBV28', 'TRBV9', 'TRBV25-1', 'TRBV20-1', 'TRBV14',
       'TRBV29-1', 'TRBV20', 'TRBV1', 'TRBV13', 'TRBV3', 'TRBV6-3',
       'TRBV23', 'TRBV6-2', 'TRBV8', 'TRBV15', 'TRBV7-9', 'TRBV19',
       'TRBV5-4', 'TRBV30', 'TRBV6-1', 'TRBV11-3', 'TRBV11-2', 'TRBV4-1',
       'TRBV23-1', 'TRBV3-1', 'TRBV6-6', 'TRBV10-3', 'TRBV12-4',
       'TRBV12-3', 'TRBV5-1', 'TRBV6-4', 'TRBV6-5', 'TRBV4-3', 'TRBV10-1',
       'TRBV7-2', 'TRBV10-2', 'TRBV27', 'TRBV7-6', 'TRBV5-8', 'TRBV18',
       'TRBV4-2', 'TRBV5-6', 'TRBV1-1', 'TRBV2-1', 'TRBV9-1', 'TRBV19*02',
       'TRBV7-1', 'TRBV9-2', 'TRBV6-4*02', 'TRBV27*01', 'TRBV20-1*01',
       'TRBV25-1*01', 'TRBV7-3*01', 'TRBV12-4*01', 'TRBV3-1*01',
       'TRBV24-1*01', 'TRBV6-2*01', 'TRBV19*01', 'TRBV28*01',
       'TRBV6-5*01', 'TRBV5-4*01', 'TRBV7-9*01', 'TRBV19*03', 'TRBV19*04',
       'TRBV4-3*01', 'TRBV7-8*01', 'TRBV4-2*01', 'TRBV22*01',
       'TRBV26-2*01', 'TRBV3*01'

In [88]:
dup_df[dup_df['beta.v'].isin(['TRBV11-1',
       'TRBV7-8', 'TRBV28', 'TRBV9', 'TRBV25-1', 'TRBV20-1', 'TRBV14',
       'TRBV29-1', 'TRBV20', 'TRBV1', 'TRBV13', 'TRBV3', 'TRBV6-3',
       'TRBV23', 'TRBV6-2', 'TRBV8', 'TRBV15', 'TRBV7-9', 'TRBV19',
       'TRBV5-4', 'TRBV30', 'TRBV6-1', 'TRBV11-3', 'TRBV11-2', 'TRBV4-1',
       'TRBV23-1', 'TRBV3-1', 'TRBV6-6', 'TRBV10-3', 'TRBV12-4',
       'TRBV12-3', 'TRBV5-1', 'TRBV6-4', 'TRBV6-5', 'TRBV4-3', 'TRBV10-1',
       'TRBV7-2', 'TRBV10-2', 'TRBV27', 'TRBV7-6', 'TRBV5-8', 'TRBV18',
       'TRBV4-2', 'TRBV5-6', 'TRBV1-1', 'TRBV2-1', 'TRBV9-1'])].__len__()

874

separate '/' in beta.v

In [89]:
clean_trbv_dict = {}
for origin_trbv in main_df['beta.v'].unique():
    print_tag = 1
    trbv = origin_trbv.strip()
    if "/" in trbv:
        if 'DV' in trbv:
            clean_trbv_dict[origin_trbv] = trbv
        elif "/TRBV" in trbv:
            clean_trbv_dict[origin_trbv] = trbv.replace("/TRBV", ",TRBV")
        else:
            clean_trbv_dict[origin_trbv] = trbv.replace("/", ",TRBV")
    else:
        clean_trbv_dict[origin_trbv] = trbv
        print_tag = 0
    if print_tag:
        print(origin_trbv, trbv, clean_trbv_dict[origin_trbv])

TRBV6-2/3 TRBV6-2/3 TRBV6-2,TRBV3
TRBV6-2/TRBV6-3 TRBV6-2/TRBV6-3 TRBV6-2,TRBV6-3
TRBV12-3/4 TRBV12-3/4 TRBV12-3,TRBV4
TRBV29/DV5*01 TRBV29/DV5*01 TRBV29/DV5*01
TRBV23/DV6*01 TRBV23/DV6*01 TRBV23/DV6*01
TRBV6-2/6-3 TRBV6-2/6-3 TRBV6-2,TRBV6-3
TRBV12-3/12-4 TRBV12-3/12-4 TRBV12-3,TRBV12-4
TRBV3-1/3-2 TRBV3-1/3-2 TRBV3-1,TRBV3-2
TRBV6-1/6-5/6-6 TRBV6-1/6-5/6-6 TRBV6-1,TRBV6-5,TRBV6-6


In [90]:
main_df['beta.v'] = main_df['beta.v'].apply(lambda x: clean_trbv_dict[x])
main_df['beta.v'].nunique()

180

In [91]:
for i in main_df['beta.v'].unique():
    if ',' in i:
        print(i)

TRBV6-2,TRBV3
TRBV6-2,TRBV6-3
TRBV12-3,TRBV4
TRBV12-3,TRBV12-4
TRBV3-1,TRBV3-2
TRBV6-1,TRBV6-5,TRBV6-6


In [92]:
clean_trbv_dict = {'TRBV6-2,TRBV3':'TRBV6-2,TRBV6-3', 'TRBV12-3,TRBV4':'TRBV12-3,TRBV12-4'}
main_df['beta.v'] = main_df['beta.v'].replace(clean_trbv_dict)
main_df['beta.v'].nunique()

178

In [113]:
split_df = main_df['beta.v'].str.split(',', expand=True)
stacked_df = split_df.stack().reset_index(level=1, drop=True).to_frame(name='beta.v')
main_df = main_df.drop(['beta.v'], axis=1).join(stacked_df).reset_index(drop=True)
len(main_df)

174674

In [115]:
import re
def check_uppercase(text):
    if text == '(UNK)':
        return True
    else:
        pattern = "^[A-Z]+$"
        return bool(re.match(pattern, text))

main_df['check'] = main_df['beta.vseq'].apply(check_uppercase)
print(main_df[main_df['check']==False]['beta.vseq'])

main_df['check'] = main_df['alpha.vseq'].apply(check_uppercase)
print(main_df[main_df['check']==False]['alpha.vseq'])

Series([], Name: beta.vseq, dtype: object)
Series([], Name: alpha.vseq, dtype: object)


In [118]:
main_df = main_df.drop_duplicates(ignore_index=True)
print(f"number of duplicated data: {len(main_df)}")

savepath = to_your_path + "TCRdata/collect/"
savename = "main_dataset_tcr_phla_cleaned.csv"
save_csv(main_df, savepath, savename)

number of duplicated data: 174673
File /data/lujd/TCRdata/collect/main_dataset_tcr_phla_cleaned.csv has been successfully saved.


In [120]:
len(main_df[main_df['alpha.vseq']!='(UNK)']), len(main_df[main_df['beta.vseq']!='(UNK)'])

(352, 383)